#  Banca Trinacria – Data Generator

**Generatore di dati bancari fittizi ma realistici**  
Progetto portfolio – *Vincenzo Alesi, Data Analyst*

---

##  Obiettivi del progetto
- Simulare un ecosistema bancario completo
- Dati realistici per analisi, SQL e BI
- Conformità a standard italiani (CF, IBAN, ecc.)

##  Volumi generati
- 50.000 clienti (Sicilia)
- 75.000 conti correnti
- 10M transazioni / anno
- 15.000 prestiti (con NPL)
- Sistema di fraud detection

##  Import librerie e setup ambiente

- Librerie standard Python
- Seed casuali per riproducibilità

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
import hashlib
from typing import List, Dict, Tuple
import json
import os
from pathlib import Path

np.random.seed(42)
random.seed(42)

## Configurazione parametri di generazione

Classe `Config`:
- volumi
- date
- parametri bancari
- directory di output

In [2]:
class Config:
    """Parametri configurabili per generazione dati"""
    
    NUM_CLIENTI = 50000
    NUM_FILIALI = 35
    NUM_DIPENDENTI = 350
    RATIO_CONTI_CLIENTE = 1.5
    NUM_TRANSAZIONI_ANNO = 10_000_000
    NUM_PRESTITI = 15000
    
    DATA_INIZIO = datetime(2015, 1, 1)
    DATA_FINE = datetime(2024, 12, 31)
    
    NPL_RATIO = 0.08
    FRAUD_RATE = 0.002
      
    # Percorso base del progetto
    BASE_DIR = Path(r"C:\Users\Vincenzo\Desktop\Banca\Python")

    # Cartella output CSV
    OUTPUT_DIR = BASE_DIR / "data" / "csv_output"

    @classmethod
    def setup_directories(cls):
        cls.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
Config.setup_directories()

##  Dati territoriali – Sicilia

- Province con pesi realistici
- Comuni principali

In [4]:
PROVINCE_SICILIA = {
    'PA': {'nome': 'Palermo', 'peso': 0.30},
    'CT': {'nome': 'Catania', 'peso': 0.25},
    'ME': {'nome': 'Messina', 'peso': 0.15},
    'SR': {'nome': 'Siracusa', 'peso': 0.10},
    'TP': {'nome': 'Trapani', 'peso': 0.08},
    'AG': {'nome': 'Agrigento', 'peso': 0.07},
    'RG': {'nome': 'Ragusa', 'peso': 0.03},
    'CL': {'nome': 'Caltanissetta', 'peso': 0.02}
}

COMUNI_PRINCIPALI = {
    'PA': ['Palermo', 'Bagheria', 'Monreale'],
    'CT': ['Catania', 'Acireale'],
    'ME': ['Messina', 'Milazzo'],
    'SR': ['Siracusa', 'Noto'],
    'TP': ['Trapani', 'Mazara del Vallo', 'Marsala'],
    'AG': ['Agrigento', 'Menfi'],
    'RG': ['Ragusa', 'Modica'],
    'CL': ['Caltanissetta', 'Gela']
}

##  Dati anagrafici

- Nomi e cognomi
- Professioni
- Categorie e tipi di transazione

In [5]:
NOMI_ITALIANI = [
    'Giuseppe', 'Maria', 'Antonio', 'Anna', 'Francesco', 'Rosa', 'Salvatore',
    'Angela', 'Vincenzo', 'Giovanna', 'Luigi', 'Teresa', 'Giovanni', 'Giuseppina',
    'Michele', 'Carmela', 'Pietro', 'Caterina', 'Domenico', 'Francesca',
    'Antonino', 'Lucia', 'Rosario', 'Stefania', 'Calogero', 'Rosaria',
    'Marco', 'Daniela', 'Andrea', 'Laura', 'Alessandro', 'Alessandra',
    'Matteo', 'Chiara', 'Davide', 'Valentina', 'Simone', 'Martina'
]

COGNOMI_SICILIANI = [
    'Alesi', 'Asaro', 'Libasci', 'Giarraputo', 'Zinna', 'Borsellino', 'Sutera'
    'Russo', 'Messina', 'Caruso', 'Lombardo', 'Marino', 'Greco', 'Bruno',
    'Gallo', 'Rizzo', 'Ferrara', 'Romano', 'Vitale', 'Giordano', 'Santoro',
    'Costa', 'De Luca', 'Ferrera', 'Fontana', 'Leone', 'Longo', 'Marchese',
    'Martino', 'Palermo', 'Parisi', 'Puglisi', 'Randazzo', 'Rizza', 'Salerno',
    'Sardo', 'Scuderi', 'Trovato', 'Valenti', 'Barbera', 'Bella', 'Castagna',
    'Catalano', 'Cavallaro', 'Coco', 'Corsaro', 'D\'Angelo', 'Fazio',
    'Fichera', 'Fiore', 'Gagliano', 'Grasso', 'La Rosa', 'Marletta',
    'Mazza', 'Nicolosi', 'Orlando', 'Pappalardo', 'Piana', 'Platania',
    'Ragusa', 'Rapisarda', 'Riccobono', 'Rossitto', 'Sciacca', 'Torrisi'
]

PROFESSIONI = [
    'Impiegato', 'Operaio', 'Insegnante', 'Commerciante', 'Artigiano',
    'Medico', 'Avvocato', 'Ingegnere', 'Pensionato', 'Studente',
    'Infermiere', 'Architetto', 'Geometra', 'Ragioniere', 'Farmacista',
    'Agente immobiliare', 'Imprenditore', 'Libero professionista',
    'Agricoltore', 'Commercialista', 'Manager', 'Consulente', 'Dirigente'
]

CATEGORIE_TRANSAZIONE = [
    'Stipendio', 'Pensione', 'Bonifico', 'Addebito utenze', 'Spesa alimentare',
    'Carburante', 'Ristorante', 'Shopping', 'Salute', 'Istruzione',
    'Viaggi', 'Intrattenimento', 'Assicurazione', 'Affitto', 'Mutuo',
    'Tasse', 'Donazione', 'Investimento', 'Risparmio', 'Prelievo ATM',
    'Pagamento POS', 'Bolletta telefono', 'Bolletta gas', 'Bolletta luce',
    'Internet', 'Palestra', 'Abbonamenti', 'Altro'
]

TIPI_TRANSAZIONE = [
    {'codice': 'BON', 'descrizione': 'Bonifico SEPA', 'categoria': 'Bancaria', 'commissione': 1.50},
    {'codice': 'RID', 'descrizione': 'Addebito diretto', 'categoria': 'Bancaria', 'commissione': 0.00},
    {'codice': 'ATM', 'descrizione': 'Prelievo bancomat', 'categoria': 'Bancaria', 'commissione': 0.00},
    {'codice': 'POS', 'descrizione': 'Pagamento POS', 'categoria': 'Commerciale', 'commissione': 0.00},
    {'codice': 'F24', 'descrizione': 'Pagamento F24', 'categoria': 'Bancaria', 'commissione': 0.00},
    {'codice': 'MAV', 'descrizione': 'Pagamento MAV', 'categoria': 'Bancaria', 'commissione': 1.00},
    {'codice': 'CBL', 'descrizione': 'Bollettino postale', 'categoria': 'Bancaria', 'commissione': 2.00},
    {'codice': 'SDD', 'descrizione': 'Addebito SEPA', 'categoria': 'Bancaria', 'commissione': 0.00},
    {'codice': 'CHK', 'descrizione': 'Assegno', 'categoria': 'Bancaria', 'commissione': 1.50},
    {'codice': 'TRF', 'descrizione': 'Giroconto', 'categoria': 'Bancaria', 'commissione': 0.00}
]

##  Utility functions

Funzioni di supporto per:
- Codice fiscale
- IBAN
- Email
- Telefono
- Credit score


In [6]:
def genera_codice_fiscale() -> str:
    """Genera un codice fiscale italiano fittizio ma formalmente valido"""
    consonanti = 'BCDFGHJKLMNPQRSTVWXYZ'
    vocali = 'AEIOU'
    cifre = '0123456789'
    
    # 6 caratteri cognome/nome (simulati)
    cf = ''.join(random.choices(consonanti, k=6))
    # 2 cifre anno
    cf += ''.join(random.choices(cifre, k=2))
    # 1 lettera mese
    cf += random.choice('ABCDEHLMPRST')
    # 2 cifre giorno
    cf += ''.join(random.choices(cifre, k=2))
    # 4 caratteri comune (simulato)
    cf += random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    cf += ''.join(random.choices(cifre, k=3))
    # 1 carattere controllo
    cf += random.choice(consonanti)
    
    return cf


def genera_iban(filiale_code: str) -> str:
    """
    Genera IBAN italiano fittizio ma formalmente valido
    Formato: IT + 2 check + 1 CIN + 5 ABI + 5 CAB + 12 conto
    """
    check_digits = random.randint(10, 99)
    cin = random.choice(string.ascii_uppercase)
    abi = '03062'  # Codice ABI fittizio Banca Trinacria
    cab = filiale_code.zfill(5)  # Codice CAB dalla filiale
    conto = str(random.randint(0, 999999999999)).zfill(12)
    
    return f'IT{check_digits}{cin}{abi}{cab}{conto}'


def genera_email(nome: str, cognome: str) -> str:
    """Genera email realistica"""
    domini = ['gmail.com', 'libero.it', 'yahoo.it', 'outlook.com', 'hotmail.it', 'alice.it']
    separatori = ['.', '_', '']
    
    nome_clean = nome.lower().replace("'", "")
    cognome_clean = cognome.lower().replace("'", "")
    
    sep = random.choice(separatori)
    dominio = random.choice(domini)
    
    # Possibilità di aggiungere numeri
    if random.random() < 0.3:
        numero = random.randint(1, 999)
        return f'{nome_clean}{sep}{cognome_clean}{numero}@{dominio}'
    
    return f'{nome_clean}{sep}{cognome_clean}@{dominio}'


def genera_telefono() -> str:
    """Genera numero telefono italiano mobile"""
    prefissi = ['320', '328', '329', '330', '331', '333', '334', '335', '336', '338', '339', '340', '342', '345', '346', '347', '348', '349']
    prefisso = random.choice(prefissi)
    numero = ''.join([str(random.randint(0, 9)) for _ in range(7)])
    return f'+39{prefisso}{numero}'


def calcola_credit_score(reddito: float, eta: int, professione: str) -> int:
    """
    Calcola credit score basato su fattori realistici
    Range: 300-850 (FICO-style)
    """
    base_score = 500
    
    # Fattore reddito (max +150)
    if reddito > 80000:
        base_score += 150
    elif reddito > 50000:
        base_score += 100
    elif reddito > 30000:
        base_score += 50
    elif reddito > 20000:
        base_score += 20
    
    # Fattore età (stabilità, max +100)
    if 35 <= eta <= 55:
        base_score += 100
    elif 25 <= eta < 35 or 55 < eta <= 65:
        base_score += 50
    elif eta > 65:
        base_score += 30
    
    # Fattore professione (max +50)
    professioni_stabili = ['Medico', 'Avvocato', 'Ingegnere', 'Insegnante', 'Impiegato', 'Dirigente']
    if professione in professioni_stabili:
        base_score += 50
    elif professione == 'Pensionato':
        base_score += 40
    
    # Variazione random ±50
    base_score += random.randint(-50, 50)
    
    # Clamp tra 300-850
    return max(300, min(850, base_score))


def genera_descrizione_transazione(categoria: str, importo: float) -> str:
    """Genera descrizione realistica per transazione"""
    descrizioni = {
        'Stipendio': ['Accredito stipendio', 'Bonifico stipendio mensile', 'Emolumenti'],
        'Pensione': ['Accredito pensione INPS', 'Pensione mensile'],
        'Bonifico': ['Bonifico a', 'Trasferimento fondi', 'Pagamento fattura'],
        'Addebito utenze': ['Bolletta energia elettrica', 'Bolletta gas', 'Bolletta acqua'],
        'Spesa alimentare': ['Supermercato COOP', 'Lidl', 'Eurospin', 'Carrefour', 'MD'],
        'Carburante': ['Rifornimento ENI', 'IP Carburante', 'Q8', 'Tamoil'],
        'Ristorante': ['Ristorante', 'Pizzeria', 'Trattoria', 'Bar'],
        'Shopping': ['Amazon', 'Zalando', 'Centro commerciale', 'Negozio abbigliamento'],
        'Prelievo ATM': ['Prelievo contante ATM'],
        'Pagamento POS': [f'Pagamento POS']
    }
    
    if categoria in descrizioni:
        base = random.choice(descrizioni[categoria])
        if categoria == 'Bonifico':
            base += f' {random.choice(COGNOMI_SICILIANI)}'
        return base
    
    return f'Operazione {categoria}'

##  Generatori entità base

- Filiali
- Dipendenti
- Clienti

In [7]:
def genera_filiali(num_filiali: int = Config.NUM_FILIALI) -> pd.DataFrame:
    """Genera rete filiali Sicilia"""
    print(f"Generazione {num_filiali} filiali...")
    
    filiali = []
    filiale_id = 1
    
    # Distribuzione per provincia
    for provincia, info in PROVINCE_SICILIA.items():
        num_filiali_provincia = max(1, int(num_filiali * info['peso']))
        comuni = COMUNI_PRINCIPALI[provincia]
        
        for i in range(num_filiali_provincia):
            comune = random.choice(comuni)
            tipo = 'Principale' if i == 0 else random.choice(['Distaccata', 'Distaccata', 'Sportello'])
            
            filiali.append({
                'filiale_id': filiale_id,
                'codice_filiale': f'BT{provincia}{str(filiale_id).zfill(3)}',
                'nome_filiale': f'Banca Trinacria - {comune}',
                'indirizzo': f'Via {random.choice(["Vittorio Emanuele", "Roma", "Umberto I", "Garibaldi"])} {random.randint(1, 150)}',
                'comune': comune,
                'provincia': provincia,
                'cap': f'9{random.randint(0, 9)}{random.randint(100, 999)}',
                'telefono': f'091{random.randint(1000000, 9999999)}',
                'email': f'{comune.lower().replace(" ", "")}@bancatrinacria.it',
                'tipo_filiale': tipo,
                'numero_dipendenti': random.randint(3, 25) if tipo == 'Principale' else random.randint(2, 8),
                'data_apertura': Config.DATA_INIZIO + timedelta(days=random.randint(0, 365*5)),
                'stato': 'Attiva',
                'created_at': datetime.now(),
                'updated_at': datetime.now()
            })
            
            filiale_id += 1
    
    df = pd.DataFrame(filiali)
    print(f"✓ {len(df)} filiali generate")
    return df

In [8]:
df_filiali = genera_filiali()
df_filiali.to_csv(Config.OUTPUT_DIR / 'filiali.csv', index=False, encoding='utf-8-sig')

Generazione 35 filiali...
✓ 32 filiali generate


In [9]:
def genera_dipendenti(df_filiali: pd.DataFrame, num_dipendenti: int = Config.NUM_DIPENDENTI) -> pd.DataFrame:
    """Genera personale bancario"""
    print(f"Generazione {num_dipendenti} dipendenti...")
    
    dipendenti = []
    
    ruoli_dist = {
        'Direttore': 0.10,
        'Gestore': 0.25,
        'Operatore': 0.40,
        'Cassiere': 0.25
    }
    
    for i in range(num_dipendenti):
        filiale = df_filiali.sample(n=1).iloc[0]
        nome = random.choice(NOMI_ITALIANI)
        cognome = random.choice(COGNOMI_SICILIANI)
        ruolo = np.random.choice(list(ruoli_dist.keys()), p=list(ruoli_dist.values()))
        
        dipendenti.append({
            'dipendente_id': i + 1,
            'matricola': f'MAT{str(i+1).zfill(6)}',
            'nome': nome,
            'cognome': cognome,
            'email': f'{nome.lower()}.{cognome.lower()}@bancatrinacria.it',
            'telefono': genera_telefono(),
            'filiale_id': filiale['filiale_id'],
            'ruolo': ruolo,
            'data_assunzione': Config.DATA_INIZIO + timedelta(days=random.randint(0, 3650)),
            'data_cessazione': None if random.random() > 0.05 else Config.DATA_FINE - timedelta(days=random.randint(0, 365)),
            'stato': 'Attivo' if random.random() > 0.05 else 'Cessato',
            'created_at': datetime.now(),
            'updated_at': datetime.now()
        })
    
    df = pd.DataFrame(dipendenti)
    print(f"✓ {len(df)} dipendenti generati")
    return df

In [10]:
df_dipendenti = genera_dipendenti(df_filiali)
df_dipendenti.to_csv(Config.OUTPUT_DIR / "dipendenti.csv", index=False, encoding="utf-8-sig")

Generazione 350 dipendenti...
✓ 350 dipendenti generati


In [11]:
def genera_clienti(df_filiali: pd.DataFrame, df_dipendenti: pd.DataFrame, 
                   num_clienti: int = Config.NUM_CLIENTI) -> pd.DataFrame:
    """Genera anagrafica clienti con distribuzione realistica"""
    print(f"Generazione {num_clienti} clienti...")
    
    clienti = []
    gestori = df_dipendenti[df_dipendenti['ruolo'] == 'Gestore']['dipendente_id'].tolist()
    
    for i in range(num_clienti):
        if (i + 1) % 10000 == 0:
            print(f"  Progresso: {i+1}/{num_clienti}")
        
        nome = random.choice(NOMI_ITALIANI)
        cognome = random.choice(COGNOMI_SICILIANI)
        
        # Età con distribuzione realistica (18-90 anni, picco 30-60)
        eta = int(np.random.normal(45, 15))
        eta = max(18, min(90, eta))
        data_nascita = datetime.now() - timedelta(days=eta*365)
        
        # Provincia e filiale
        provincia = np.random.choice(
            list(PROVINCE_SICILIA.keys()),
            p=[info['peso'] for info in PROVINCE_SICILIA.values()]
        )
        filiali_provincia = df_filiali[df_filiali['provincia'] == provincia]
        filiale = filiali_provincia.sample(n=1).iloc[0]
        comune = random.choice(COMUNI_PRINCIPALI[provincia])
        
        # Reddito con distribuzione log-normale
        reddito_base = np.random.lognormal(10.3, 0.5)  # Media ~30K, range 10K-150K
        reddito = max(8000, min(500000, reddito_base))
        
        professione = random.choice(PROFESSIONI)
        credit_score = calcola_credit_score(reddito, eta, professione)
        
        # Segmentazione
        if reddito >= 100000:
            segmento = 'Private'
        elif professione in ['Imprenditore', 'Libero professionista', 'Manager']:
            segmento = 'Business'
        else:
            segmento = 'Retail'
        
        clienti.append({
            'cliente_id': i + 1,
            'codice_fiscale': genera_codice_fiscale(),
            'nome': nome,
            'cognome': cognome,
            'data_nascita': data_nascita.date(),
            'genere': random.choice(['M', 'F']),
            'email': genera_email(nome, cognome),
            'telefono': genera_telefono(),
            'indirizzo': f'Via {random.choice(["Roma", "Vittorio Emanuele", "Garibaldi", "Cavour"])} {random.randint(1, 200)}',
            'comune': comune,
            'provincia': provincia,
            'cap': f'9{random.randint(0, 9)}{random.randint(100, 999)}',
            'segmento': segmento,
            'reddito_annuo': round(reddito, 2),
            'professione': professione,
            'credit_score': credit_score,
            'stato_cliente': 'Attivo' if random.random() > 0.02 else random.choice(['Sospeso', 'Chiuso']),
            'filiale_id': filiale['filiale_id'],
            'gestore_id': random.choice(gestori) if gestori else None,
            'data_acquisizione': Config.DATA_INIZIO + timedelta(days=random.randint(0, 3650)),
            'created_at': datetime.now(),
            'updated_at': datetime.now()
        })
    
    df = pd.DataFrame(clienti)
    print(f"✓ {len(df)} clienti generati")
    print(f"  Segmentazione: {df['segmento'].value_counts().to_dict()}")
    return df

In [12]:
df_clienti = genera_clienti(df_filiali, df_dipendenti)
df_clienti.to_csv(Config.OUTPUT_DIR / "clienti.csv", index=False, encoding="utf-8-sig")

Generazione 50000 clienti...
  Progresso: 10000/50000
  Progresso: 20000/50000
  Progresso: 30000/50000
  Progresso: 40000/50000
  Progresso: 50000/50000
✓ 50000 clienti generati
  Segmentazione: {'Retail': 43190, 'Business': 6439, 'Private': 371}


##  Prossimi step

- Generazione conti correnti
- Transazioni (con stagionalità)
- Prestiti e NPL
- Fraud detection
- Analisi KPI e dashboard
